### Instalar bibliotecas

- Necessário baixar o Ollama no computador: https://ollama.com/download
- Após baixar o ollama, escreva o seguinte comando no terminal `ollama pull nomic-embed-text`.

In [ ]:
!pip install --upgrade pip
!pip install "pymongo[srv]"==3.12
!pip install python-dotenv
!pip install langchain-mongodb
!pip install langchain-groq
!pip install langchain_community

- https://www.mongodb.com/pt-br/docs/atlas/atlas-vector-search/ai-integrations/langchain/get-started/
- https://www.mongodb.com/pt-br/docs/atlas/atlas-vector-search/ai-integrations/langchain/

In [ ]:
import os, pymongo
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_mongodb import MongoDBAtlasVectorSearch
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pymongo import MongoClient
from pymongo.operations import SearchIndexModel


from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from os import getenv
from dotenv import load_dotenv

from langchain.embeddings import OllamaEmbeddings
from langchain_groq import ChatGroq

from pprint import pprint

load_dotenv('dev.env')

### Conectar com o banco

- Acesse o arquivo `dev.env_exemplo`.
- Inclua seus dados de conexão do mongoDB
- Inclua a chave da API do grop
- Renomear o nome do arquivo para `dev.env`

- GROQ: https://console.groq.com/keys
- MongoDB: https://cloud.mongodb.com/

In [3]:
USUARIO = getenv('USUARIO')
SENHA = getenv('SENHA')
NOME_CLUSTER = getenv('NOME_CLUSTER')
HOSTNAME = getenv('HOSTNAME')

In [ ]:
uri = f"mongodb+srv://{USUARIO}:{SENHA}@{NOME_CLUSTER}.{HOSTNAME}.mongodb.net"

client = MongoClient(uri, server_api=ServerApi('1'))

try:
    client.admin.command('ping')
    print("Conexão estabelecida!")
except Exception as e:
    print(e)

### Processar arquivo PDF

- Usei um PDF de história como exemplo para processar o conteudo e enviar para o banco

In [5]:
link_pdf = "https://www.caesp.com.br/libwww/colegios/uploads/uploadsMateriais/07022023143038A-Historia-do-Mundo-Para-Quem-Tem-Pressa-by-Emma-Marriott-z-lib.org_.epub_.pdf"

loader = PyPDFLoader(link_pdf)
data = loader.load()

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
docs = text_splitter.split_documents(data)

In [ ]:
# Processa os textos e cria embeddings de cada um

vector_store = MongoDBAtlasVectorSearch.from_connection_string(
  connection_string = uri,
  namespace = "langchain_db.test",
  embedding = OllamaEmbeddings(model="nomic-embed-text"),
  index_name = "vector_index"
)

In [ ]:
vector_store.add_documents(documents=docs)

In [ ]:
vector_store.create_vector_search_index(
   dimensions = 768, # As dimensões dos embeddings vetoriais a serem indexados
   filters = [ "page_label" ]
)

In [ ]:
# Fazendo testes de pesquisa

query = "china antiga"
results = vector_store.similarity_search(query)
pprint(results)

### Criação do RAG para pesquisar dados no banco

In [11]:
model = "llama3-70b-8192"
model1 = "meta-llama/llama-4-maverick-17b-128e-instruct"
llm = ChatGroq(model=model, temperature=0)

#### Configuração do modelo para criar perguntas

In [ ]:
retriever = vector_store.as_retriever(
   search_type = "similarity",
   search_kwargs = { "k": 10,
                     "score_threshold": 0.65 }
)

# Define a prompt template
template = """
   Crie 1 questão de multipla escolha em português, com 5 opções de resposta. No final da questão coloque a alternativa correta. Essa questão deve ser criada com base no contexto a abaixo.
   {contexto}
   Topico: {topico}
"""

prompt = PromptTemplate.from_template(template)

chain = (
   { "contexto": retriever, "topico": RunnablePassthrough()}
   | prompt
   | llm
   | StrOutputParser()
)

#### Testando o modelo

In [ ]:
# Prompt
questao = "Estados Unidos"
answer = chain.invoke(questao)

print("Questão: " + questao)
print("Resposta: " + answer)


documents = retriever.invoke(questao)
print("\nFonte dos documentos:")
pprint(documents)